# Customer Insights Analysis In Retail CRM

### Data Preparation

In [25]:
# Importing necessary required libraries for analysis

import pandas as pd
import numpy as np

In [26]:
# Reading source file

df_DCR = pd.read_excel(r"F:\Data Science_Projects & Files\Projects\Customer_Insights_Analysis_In_Retail_CRM\Analysis\Data_Sources\Decathlon_Customer Responses.xlsx")

In [27]:
# Removing Duplicates

df_DCR = df_DCR.drop_duplicates(subset='Name', keep='first')
df_DCR = df_DCR.drop_duplicates(subset='Timestamp', keep='first')
df_DCR.columns = df_DCR.columns.str.strip()

In [28]:
# Splitting Name Column to 3 Columns and Filling Null Values with Blank

df_DCR[['first_name', 'middle_name', 'last_name']] = df_DCR['Name'].str.split(' ', expand=True)
df_DCR = df_DCR.fillna('')

# Removing "." from all Names and concating all 3 Names columns by creating new column 'Full Name'

df_DCR['first_name'] = df_DCR['first_name'].str.rstrip('.')
df_DCR['middle_name'] = df_DCR['middle_name'].str.rstrip('.')
df_DCR['last_name'] = df_DCR['last_name'].str.rstrip('.')
df_DCR['full_name'] = df_DCR['first_name'] + ' ' + df_DCR['middle_name'] + ' ' + df_DCR['last_name']

# Removing all other Names columns

df_DCR.drop(columns=['Name', 'first_name', 'middle_name', 'last_name'], inplace=True)

In [29]:
# Splitting Date and Time with 12 Hour Format and removing Time Zone and clearing whitespaces

df_DCR[['date', 'time_12hr']] = df_DCR['Timestamp'].str.split(' ', n=1, expand=True)
df_DCR['time_12hr'] = df_DCR['time_12hr'].str.replace('GMT.*', '', regex=True).str.strip()

# Converting 12 Hour Time Format to 24 Hour Time Format

df_DCR['time'] = pd.to_datetime(df_DCR['time_12hr'], format = '%I:%M:%S %p', errors='coerce').dt.strftime('%H:%M:%S')

# Adding Date and Time into one columne date_time and dropping columns date and time columns

df_DCR['date_time'] = pd.to_datetime(df_DCR['date'] + ' ' + df_DCR['time'], errors='coerce')
df_DCR.drop(columns=['Timestamp', 'time_12hr', 'date', 'time'], inplace=True)

In [30]:
# Renaming and Sorting Columns

columns_reorder = [
                'date_time',
                'full_name',
                'Gender',	
                'Age of Customer?',
                'Frequency of visit to the store?',	
                'I am comfortable in using online medium like email, website etc. for queries and complaints',
                'I prefer to go home and complain about my issues online',
                'Do you feel that issues of customer are addressed in the newer version of the products?',
                'Before purchasing do you go through the product details online and analyze it?',	
                'I have felt being deprived of information about a product when asked',
                'Do you feel that communicating complaints online is safe, secure and without information loss?',
                'I have faced hurdles while raising complaints in person',
                'I get instant response to my complaints raised through online',
                ]

columns_rename = {
                'Gender' : 'gender',
                'full_name' : 'name',
                'Age of Customer?' : 'age_group',
                'Frequency of visit to the store?' : 'visit_frequency',	
                'I am comfortable in using online medium like email, website etc. for queries and complaints' : 'online_query_comfort',
                'I prefer to go home and complain about my issues online' : 'online_compliant_preference',
                'Do you feel that issues of customer are addressed in the newer version of the products?' : 'resolution_in_new_products',
                'Before purchasing do you go through the product details online and analyze it?' : 'pre-purchase_research',	
                'I have felt being deprived of information about a product when asked' : 'information_deprivation',
                'Do you feel that communicating complaints online is safe, secure and without information loss?' : 'online_compliant_safety',
                'I have faced hurdles while raising complaints in person' : 'in-person_complaint_issues',
                'I get instant response to my complaints raised through online' : 'instant_online_response'
                }

df_DCR = df_DCR.reindex(columns=columns_reorder).rename(columns=columns_rename).reset_index().drop(columns='index')

In [31]:
# Applying Title Case to all the Text Columns

df_DCR = df_DCR.apply(lambda col: col.str.title() if col.dtype == "object" else col)

# Coverting Categorical Columns to type category to save memory

columns_to_convert = df_DCR.columns.difference(['date_time', 'name'])
df_DCR[columns_to_convert] = df_DCR[columns_to_convert].astype('category')
df_DCR

,date_time,name,gender,age_group,visit_frequency,online_query_comfort,online_compliant_preference,resolution_in_new_products,pre-purchase_research,information_deprivation,online_compliant_safety,in-person_complaint_issues,instant_online_response
0,2022-01-02 11:43:11,Bhopesh,Male,21-30,Often,Agree,Agree,No,Yes,Disagree,Yes,Agree,Agree
1,2022-01-02 11:45:36,Bhuvaneshwar T,Male,21-30,Rarely,Agree,Disagree,No,Yes,Strongly Disagree,Yes,Disagree,Agree
2,2022-01-02 11:54:25,Santhosh P,Male,21-30,Sometimes,Agree,Agree,Yes,Yes,Disagree,Yes,Neutral,Neutral
3,2022-01-02 11:57:41,Chithra,Female,41-50,Sometimes,Strongly Agree,Agree,Yes,No,Disagree,Yes,Disagree,Strongly Agree
4,2022-01-02 12:00:41,Rohit G,Male,21-30,New Customer,Strongly Agree,Agree,Yes,Yes,Neutral,Yes,Disagree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,2022-02-02 18:13:35,Bhavya Ilango S,Female,31-40,New Customer,Strongly Disagree,Agree,No,No,Agree,No,Disagree,Strongly Agree
2296,2022-02-02 10:48:29,K Pranith,Male,31-40,Often,Neutral,Strongly Disagree,No,Yes,Strongly Disagree,No,Strongly Agree,Disagree
2297,2022-02-02 11:35:03,G Gauri,Female,15-20,Often,Neutral,Strongly Disagree,Yes,Yes,Agree,No,Disagree,Strongly Disagree
2298,2022-02-02 17:25:58,A Prabhubalaji,Male,15-20,New Customer,Disagree,Neutral,Yes,Yes,Strongly Disagree,No,Agree,Neutral


In [32]:
# Exporting to .feather format

df_DCR.to_feather(r"F:\Data Science_Projects & Files\Projects\Customer_Insights_Analysis_In_Retail_CRM\Analysis\Output_Data\DCR_cleaned_data.feather")